## MLflow Local Deployment Demo

In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

### Star the Mlflow Server 

Anaconda PS CLI: mlflow server
Default URI 127.0.0.1 and Port 5000

## Load training data and train a simple model

For our quickstart, we're going to be using the familiar iris dataset that is included in scikit-learn. Following the split of the data, we're going to train a simple logistic regression classifier on the training data and calculate some error metrics on our holdout test data. 

In [15]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model hyperparameters
params = {"solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate accuracy as a target loss metric
accuracy = accuracy_score(y_test, y_pred)

/home/bla/.local/share/virtualenvs/mlops-0su7QrrH/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


### Set the Tracking URI

Default is artifacts:/ but doesn't work for local deployment (at least, in Windows)
Thus we set a local path for tracking

In [16]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

### Start (or create) an experiment

Important: the experiment (and model, later) won't appear in the Mlflow UI, because they are tracked locally!

In [17]:
from pathlib import Path

experiment = mlflow.set_experiment("test5")

## Log the model, hyperparameters, and loss metrics to MLflow.

In [18]:
# Start an MLflow run
with mlflow.start_run() as run:
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train
    )

2024/09/06 09:41:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run sedate-turtle-687 at: http://127.0.0.1:5000/#/experiments/707500131727696303/runs/2d7f51ad66dc411f8c89cbe06ac0317b.
2024/09/06 09:41:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/707500131727696303.


In [19]:
model_uri = f"runs:/{run.info.run_id}/sklearn-model"
mv = mlflow.register_model(model_uri, "ClassificationModel")
print(f"Name: {mv.name}")
print(f"Version: {mv.version}")

Successfully registered model 'ClassificationModel'.
2024/09/06 09:41:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ClassificationModel, version 1


Name: ClassificationModel
Version: 1


Created version '1' of model 'ClassificationModel'.


In [26]:
experiment
experiment.experiment_id


mlflow.get_run(run.info.run_id)

<Run: data=<RunData: metrics={'accuracy': 1.0}, params={'max_iter': '1000',
 'multi_class': 'auto',
 'random_state': '8888',
 'solver': 'lbfgs'}, tags={'Training Info': 'Basic LR model for iris data',
 'mlflow.log-model.history': '[{"run_id": "2d7f51ad66dc411f8c89cbe06ac0317b", '
                             '"artifact_path": "iris_model", '
                             '"utc_time_created": "2024-09-06 '
                             '07:41:02.289577", "flavors": {"python_function": '
                             '{"model_path": "model.pkl", "predict_fn": '
                             '"predict", "loader_module": "mlflow.sklearn", '
                             '"python_version": "3.10.12", "env": {"conda": '
                             '"conda.yaml", "virtualenv": "python_env.yaml"}}, '
                             '"sklearn": {"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.5.1", '
                             '"serialization_format": "cloudpickle

In [34]:
run = mlflow.get_run(run.info.run_id)
run.info
mlflow.get_artifact_uri(run.info.run_id)

'mlflow-artifacts:/707500131727696303/e918f465d6a74a38b316ca706ec995d6/artifacts/2d7f51ad66dc411f8c89cbe06ac0317b'

In [22]:
!mlflow models serve -m runs:/2d7f51ad66dc411f8c89cbe06ac0317b/sklearn-model -p 5000

Traceback (most recent call last):
  File "/home/bla/.local/share/virtualenvs/mlops-0su7QrrH/bin/mlflow", line 8, in <module>
    sys.exit(cli())
  File "/home/bla/.local/share/virtualenvs/mlops-0su7QrrH/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/home/bla/.local/share/virtualenvs/mlops-0su7QrrH/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/home/bla/.local/share/virtualenvs/mlops-0su7QrrH/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/bla/.local/share/virtualenvs/mlops-0su7QrrH/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/bla/.local/share/virtualenvs/mlops-0su7QrrH/lib/python3.10/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)

#### Start model serving

Find the model artifact under the local tracking path
The port has to be different than that of the Mlflow Server!

Two options to run in Anaconda Power Shell CLI (both should be the same):

Anaconda PS CLI: mlflow models serve -m C:\tmp\my_tracking_file\974569135218729171\f1b24cbfb9f54bb3ba1e0f458fcd990a\artifacts\iris_model -p 5001 --no-conda

Anaconda PS CLI: mlflow models serve -m C:\tmp\my_tracking_file\974569135218729171\f1b24cbfb9f54bb3ba1e0f458fcd990a\artifacts\iris_model -p 5001  --env-manager local

### Test the model serving service

Ping is to check that is up and running

In [8]:
!curl http://127.0.0.1:5001/ping

curl: (7) Failed to connect to 127.0.0.1 port 5001 after 0 ms: Connection refused


### Score the model by sending a request

In [403]:
!curl http://127.0.0.1:5001/invocations -H "Content-Type: application/json" -d "{\"inputs\":[[0.09178,0.0,4.05,0.0]]}"

{"predictions": [1]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    55  100    20  100    35   5735  10037 --:--:-- --:--:-- --:--:-- 18333
